In [1]:
import calendar
import json
from collections import Counter
from datetime import date
from pprint import pprint
import os
from pymongo import MongoClient

In [2]:
with open(os.path.expanduser('~/.mongo/mongo.json'),'r') as config:
    mongo_uri = json.load(config)['mongo_uri']
client = MongoClient(mongo_uri)
db = client.forethought

organization_id = '440'
start = '2019-04-01T00:00:00Z'
end = '2021-06-03T00:00:00Z'
mongo_org_collection = db[str(organization_id)]
escalation = ["roadie_escalations", "ubiquity_escalation"]

In [3]:
def get_dataset():
    tickets = mongo_org_collection.find({
        'doc_type': 'ticket',
        "is_deleted": False,
        'original_item.via.channel': {'$in':['web','email', 'api']},
        'original_item.created_at': {'$gte': start, '$lte': end},
        "original_item.tags": {"$nin": escalation}
    })
    
    file_path = os.path.expanduser('~/Downloads/rdie_non_esc.json')
    with open(file_path, 'w') as f:
        count = 0
        for ticket in tickets:
            if count < 2000:
                i = ticket["original_item"]
                variable_labels = [x for x in escalation if x in i['tags']]
                j = {
                    "text_a": i["subject"],
                    "text_b": i["description"],
                    "created_at": i["created_at"],
                    "label": "no macro applied"
                    }
                    
                json.dump(j, f)
                _ = f.write('\n')
                count += 1
                if count % 100 == 0:
                    print(count)
            else:
                break

    print("total: ", count)


get_dataset()

100
200
300
400
500
600
700
800
total:  800
